# Stacked LSTM for sales prediction

Import Libraries

In [1]:
import pandas as pd
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Function to split a univariate sequence

In [2]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


Functionality for Data Preparation for Model Training




In [3]:
def data_prep(df):
  # choose a number of time steps
  n_steps = 3

  #Convert 'Sales' column to a list
  raw_seq=[]

  for i in df['Sales']:
    raw_seq.append(i)
  
  # split into samples
  X, y = split_sequence(raw_seq, n_steps)
  
  # taking all the data except last row
  X_train = X[:len(X)-1]
  y_train = y[:len(y)-1]
  
  # test sample would be data corresponding to last row
  X_test = X[len(X)-1]
  y_test = y[len(y)-1] 
  
  # reshape from [samples, timesteps] into [samples, timesteps, features]
  n_features = 1
  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
  X_test = X_test.reshape((1, n_steps, n_features))

  print('Data Preparation DONE!')
  # Build and train the model
  build_model(X_train,y_train,X_test,y_test,n_steps,n_features) 



Define Model

In [4]:
def build_model(X_train,y_train,X_test,y_test,n_steps,n_features):
  model = Sequential()
  model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
  model.add(LSTM(50, activation='relu'))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  
  print('Model Built!')

  # fit model
  model.fit(X_train, y_train, epochs=200, verbose=0)
  print('Model Trained!')
  
  #make Predictions
  print('Now making predictions')
  yhat = model.predict(X_test, verbose=0)
  print('predicted value : ' + str(yhat[0][0]))
  print('actual value : ' + str(y_test))

Read dataset using pandas

First will be quarterly sales prediction

In [5]:
df1 = pd.read_excel('/content/Sales-quarterly.xlsx', header=0, index_col=0)

In [6]:
data_prep(df1)

Data Preparation DONE!
Model Built!
Model Trained!
Now making predictions
predicted value : 210048.97
actual value : 611909.638


Second will be Monthly sales prediction

In [7]:
df2 = pd.read_excel('/content/Sales-monthly.xlsx', header=0, index_col=0)

In [8]:
data_prep(df2)

Data Preparation DONE!
Model Built!
Model Trained!
Now making predictions
predicted value : 422357.0
actual value : 513890.482
